In [1]:
from db import call_db_json
from util_func import hk_flag_checker
from release_gy3 import get_demand
import pandas as pd
from queries import hk_unreleased, hk_unreleased_warning
hk_unreleased_warning(due="order_scheduled_due", date=90, update=True)


throttle_2880_2841.pth


'[{"reference_number":"HK334582","order_scheduled_due":1691539200000,"order_number":"M240361","product_family":"SR","product_model":"15","item_description":"SR15-228LS(GK) RAIL","printed_due":1691539200000,"item_number":"300251023304300","product_g1":9.5,"product_pitch":null,"product_length":228.0,"order_quantity":6.0,"facility":"B0010 C0050 B0020 B0021 B0040 B0070 B0080 C0060 C0080 C0090 D0020 F0010 K0010 K0020","qty_string":"6","due":"08-09","mos":"M240361","qtys":"6","due_dates":"08-09","pred":1234800000,"start_due":1689897600000},{"reference_number":"HK327743","order_scheduled_due":1691107200000,"order_number":"M227922","product_family":"SHW","product_model":"27","item_description":"SHW27-2460LT RAIL","printed_due":1691107200000,"item_number":"506851005545800","product_g1":47.0,"product_pitch":60.0,"product_length":2460.0,"order_quantity":40.0,"facility":"C0010 C0020 B0080 C0040 C0060 C0080 F0010 J0020","qty_string":"10","due":"08-04","mos":"M227922 M227923 M227924 M227925","qtys":

In [ ]:
hk_unreleased.GY3_grouped.head()

In [3]:
demand, _ = get_demand(90)

In [19]:
class HkFlag:
    def __init__(self):
        self.empty = True

    def set_data(self, db_response):
        self.empty = False
        df = pd.DataFrame(db_response)
        g = df.groupby("reference_number")
        flag = g.apply(hk_flag_checker)
        self.flag = flag
        set_hk_list_string = str(flag[flag == "RBS"].index.to_list())[1:-1]
        sql = f"""
                SELECT order_number, reference_number, order_quantity, order_status from manufacturing_orders
                    WHERE order_release_code = 5
                        AND product_block_count > 0
                        AND reference_number in ({set_hk_list_string})
            """
        block_df = pd.DataFrame(call_db_json(sql))
        block_df["un_released"] = block_df["order_status"].apply(
            lambda x: True if x == 10 else False
        )
        self.block_df = block_df

    def get_data(self):
        return self.flag
    
hk_flag = HkFlag()

def get_set_flag(update=False):
    if hk_flag.empty or update:
        hk_flag_sql = """
            SELECT DISTINCT item_description, reference_number, product_family from manufacturing_orders
                WHERE order_status != 55
                    AND reference_number like 'HK%'
                    AND order_release_code = '5'
            """
        db_response = call_db_json(hk_flag_sql)
        df = pd.DataFrame(db_response)
        hk_flag.set_data(df)
    else:
        return hk_flag.get_data()
get_set_flag(True)

In [20]:
import numpy as np
block_df = hk_flag.block_df

# g = block_df.sort_values('order_number').groupby('reference_number')
# qty = g['order_quantity'].agg(lambda x: x.tolist())
# released = g['relesed'].agg(lambda x: x.tolist())
# mos = g['order_number'].agg(lambda x: x.tolist())
# t = pd.concat([mos, qty, released], axis=1)
t = block_df.sort_values('order_number').groupby('reference_number').agg({
    'order_number': list,
    'order_quantity': list,
    'un_released': list
})



In [26]:
def count_unreleased(row):    
    return sum([a * b for a, b in zip(row.order_quantity, row.un_released)])
    # print(row.un_released)
t['unreleasd_qty'] = t.apply(count_unreleased, axis=1)
t.columns = ["block_" + name for name in t.columns]
t.columns

Index(['block_order_number', 'block_order_quantity', 'block_un_released',
       'block_unreleasd_qty'],
      dtype='object')

In [28]:
def abc(row):
    return {"mos":row.block_order_number}
    
t['test'] = t.apply(abc, axis=1)
t

,block_order_number,block_order_quantity,block_un_released,block_unreleasd_qty,test
reference_number,,,,,
HK236683,"[M064032, M064050]","[9.0, 9.0]","[False, True]",9.0,"{'mos': ['M064032', 'M064050']}"
HK290670,"[M165962, M165963]","[4.0, 8.0]","[True, True]",12.0,"{'mos': ['M165962', 'M165963']}"
HK290699,"[M165036, M165037, M165038, M165039, M165040, ...","[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20....","[False, True, False, False, True, True, True, ...",240.0,"{'mos': ['M165036', 'M165037', 'M165038', 'M16..."
HK290700,"[M165072, M165073, M165074, M165075, M165076, ...","[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20....","[True, True, True, True, True, True, True, Tru...",300.0,"{'mos': ['M165072', 'M165073', 'M165074', 'M16..."
HK292395,"[M169203, M169204, M169205, M169206, M169207, ...","[4.0, 12.0, 12.0, 12.0, 12.0, 12.0, 2.0, 6.0, ...","[True, True, True, True, True, True, True, Tru...",96.0,"{'mos': ['M169203', 'M169204', 'M169205', 'M16..."
...,...,...,...,...,...
HK336462,"[M242173, M242174]","[4.0, 2.0]","[True, True]",6.0,"{'mos': ['M242173', 'M242174']}"
HK336473,[M242183],[2.0],[True],2.0,{'mos': ['M242183']}
HK336492,"[M242211, M242213]","[3.0, 3.0]","[True, True]",6.0,"{'mos': ['M242211', 'M242213']}"
